# What we tweet about when we tweet about mental health

This notebook follows the methodology described in our report in order to answer our research questions. 

_Note:_ We provid an NBViewer Notebook link to display the maps.

### Overview:

<ol start="0">
  <li><a href="#t1"><b>Data Collection &amp; Preliminary Analysis:</b></a> datasets retrieval and quick analysis to familiarize ourselves with them</li>
  <li><a href="#t2"><b><i>Spinn3r</i> Data Treatment:</b></a> 
      application of our cleaning and NLP pipeline to the $1^{st}$ dataset</li>
  <li><a href="#t3"><b><i>Spinn3r</i> Dataset Description:</b></a> 
      a first (less naive) analysis of the dataset</li>
  <li><a href="#t4"><b>Dictionary Construction</b></a> (using the <b><i>Spinn3r</i></b> dataset)<b>:</b> 
      creation (and refining) of the dictionary to be used on both datasets</li>
  <li><a href="#t5"><b><i>Spinn3r</i> Dataset Analysis:</b></a> 
      first answer to our research questions using the <i>"smaller" <b>Spinn3r</b></i> dataset</li>
  <li><a href="#t6"><b><i>twitter-swisscom</i> Data Processing Pipeline:</b></a> 
      after using the <b><i>Spinn3r</i></b> set as a proof of concept, we apply the cleaning, NLP and keyword-based selection methods to our $2^{nd}$ dataset</li>
      <li><b><a href="#t7"><i>twitter-swisscom</i> Dataset Analysis:</b></a> 
      we analyse the newly gained insights</li>
  <li><a href="#t8"><b>Conclusion</b></a></li>
</ol>

We quickly import the libraries we use in this Notebook (and define our constant values):

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import pyspark as ps
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#Part 1
import json
from pandas.io.json import json_normalize

#Part 2
import re
import unicodedata
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, TweetTokenizer

#Part 3
import pyLDAvis.gensim
from gensim import corpora, models
import networkx as nx

#Maps display
import folium
import shapely
import pysal as ps
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point
from folium.plugins import MarkerCluster
%matplotlib inline

In [ ]:
#Charts colors
colors = plt.cm.RdYlBu(np.linspace(.2, .7, 4))
colorsBar = plt.cm.PuBu(np.linspace(.2, .7, 10))

In [ ]:
#EXAMPLE DATA
EXAMPLE_PATH = 'data/example_data/'
SP_EX_TWEETS = EXAMPLE_PATH + 'spinn3r_example.json'
TS_EX_TWEETS = EXAMPLE_PATH + 'twitterswisscom_example.tsv'

In [ ]:
#SPINN3R DATASETS
SP_PATH = 'data/spinn3r_tweets/'
SP_PROC_TWEETS = SP_PATH + 'processed_tweets.pkl'

In [ ]:
#SPINN3R KEYWORDED TWEETS SUBSETS
KW_SP_PATH = SP_PATH + 'keyword_tweets/'
SP_EN_KW = KW_SP_PATH + 'english_keyworded_tweets.pkl'
SP_FR_KW = KW_SP_PATH + 'french_keyworded_tweets.pkl'
SP_DE_KW = KW_SP_PATH + 'german_keyworded_tweets.pkl'
SP_SMALL_EN_KW = KW_SP_PATH + 'small_english_keyworded_tweets.pkl'
SP_SMALL_FR_KW = KW_SP_PATH + 'small_french_keyworded_tweets.pkl'
SP_SMALL_DE_KW = KW_SP_PATH + 'small_german_keyworded_tweets.pkl'

In [ ]:
#SPINN3R LABELED TWEETS SUBSETS
LBL_SP_PATH = SP_PATH + 'labeled_tweets/'
SP_EN_LBL = LBL_SP_PATH + 'english_labeled.csv'
SP_NEW_EN_LBL = LBL_SP_PATH + 'new_english_labeled.csv'

In [ ]:
#DICTIONARIES
DICT_1 = "data/dictionaries/dict_1.csv"
DICT_2 = "data/dictionaries/dict_2.0.csv"

In [ ]:
#TWITTER-SWISSCOM DATASETS
SP_PATH = 'data/twitter-swisscom/'
TWITTERSWISSCOM_PROCESSED_TWEETS = 'data/twitter-swisscom/dataset2_english.csv'

<h2 id="t1">0. Data Collection & Preliminary Analysis</h2>

### 0.1. Data collection

We were provided two separate datasets containing Swiss tweets formated differently and containing different fields.

Our first dataset (_**Spinn3r**_) did not have any example subset. As a stand-in, we extracted the first JSON file to perform our preliminary analysis. To do this, we used the following commands after connecting to the cluster:

```bash
cluster$ hadoop fs -getmerge /datasets/swiss-tweet/harvest3r_twitter_data_01-01_0.json /buffer/example.json
local$ scp -r gaspar@iccluster060.iccluster.epfl.ch:/buffer/example.json <local-path>
```

An example dataset as well as the complet dataset of (_**twitter-swisscom**_) was retrieved from a provided _'.zip'_ file. 

### 0.2. Preliminary analysis

The goal of this preliminary analysis is to familiarize ourselves with our datasets in order to know if we need to adapt our research questions or enrich the dataset with external information. We will see (below) that the main difference between the two datasets is that the $1^{st}$ dataset (_**Spinn3r**_) only covers a span of 10 months while the data of the $2^{nd}$ dataset (_**twitter-swisscom**_) was collected over multiple years and contains more precise location information. Another difference between the datasets was the fact that the $1^{st}$ contained a column expliciting the _language_, _gender of the user_, and the _predominant sentiment_ of the tweet.

After our analysis, we intended to use **both datasets**. For this reason, we use the $1^{st}$ (smaller) dataset to write our functions and methods. On top of having results, we intended to use this as a proof of concept of our analysis pipeline to be applied to the bigger dataset.

However, due to issues with the larger dataset reducing the amount of defacto available data in the large dataset to the same as in the smal dataset, we use the second dataset for purpose of comparison of keyword co-ocurence.

#### 0.2.1. Dataset 1: _Spinn3r_

The [Spinn3r website](http://docs.spinn3r.com/?Example#content-schema) offers an elaborate description of each available field. Given the amount of data present in the cluster, we only look at a single file (corresponding to a day) to perform our preliminary analysis. This is a nested _JSON_ file. As the *read_json* function does not work well with nested _JSON_ data, we use a _JSON_ normalizer (provided in the _**Pandas.io**_ libary).

In [ ]:
with open(SP_EX_TWEETS) as data_file:    
    data = json.load(data_file)

ex_sp_df = json_normalize(data)

#rename columns for convenience
ex_sp_df.columns = [column.replace('_source.','') for column in ex_sp_df.columns]
ex_sp_df.columns

Out of all these fields, we see that the most useful columns for our analysis are:
- **main**: contains the content of the tweet (the center of our analysis).
- **published**: gives the time on which the content was posted.
- **source_location**: location of the tweet.
- **lang**: language of the tweet.
- **author_gender**: gender of the author -MALE, FEMALE, UNKNOWN-.
- **sentiment**: sentiment score of the tweet -POSITIVE, NEGATIVE, NEUTRAL-.

_Note :_ On the cluster we find additional fields, out of which we decide to use **'author_user_id'**, as means to uniquely identify an user and **'geo_point'**, containing latitude and longitude, as a way to identify user location.

In [ ]:
#defining the columns to keep
columns = ['main', 'published', 'source_spam_probability', 'source_location', 'tags', 'lang', 'sentiment',
                   'author_gender', 'source_followers', 'source_following']
ex_sp_df = ex_sp_df[columns]

**Looking at categorical data :**

In [ ]:
plt.figure(figsize=[8, 5])
ex_sp_df['source_spam_probability'].value_counts().plot(kind='pie', colors=colors)
plt.title('Spam probability in tweets')
plt.axis('equal')
plt.show()

The spam probability of the tweet set seemed very promising to clean our data, but we see that not a single tweet was labeled as spam which makes us question the accuracy of the labeling.

We now look at the values contained in the **source_location** column (which seem to be language dependent):

In [ ]:
location_occurences = ex_sp_df[ex_sp_df.lang.isin(['de', 'fr', 'en'])]['source_location'].value_counts()
print('Most frequent:\n' , location_occurences.head(), '\n')
print('Least frequent:\n' , location_occurences.tail())

We notice that:
- A lot of locations only differ in language (e.g. Switzerland, Schweiz and Suisse)
- The locations' names are not always written in languages we are interested in (e.g. สวิตเซอร์แลนด์, a location name in our list)
- A vast majority of the dataset is just located in 'Switzerland'
- Some of the tweets are not located in Switzerland

To deal with these different locations (and display them on a map) we use the **geo_point** field instead.

_Note :_ we choose to not deal with the last point mentioned under the assumption that we were provided with a dataset only containing tweets written in Switzerland.

In [ ]:
ex_sp_df.count()

When looking at the number of values of the dataset, we see that most tags associated to tweets are missing. This may indicate that tags won't be useful for our analysis, but this does not influence our research at this stage.

We also note that for some tweets, the sentiment is missing. As noted on Spinn3er, this may be due to the fact that some tweets do not contain enough linguistic information to identify the sentiment (we will filter out such tweets). However, this field is not central to our analysis, it is merely used to help us evaluate the treatment applyied to our dataset.

**Identifying potential issues :**

While this set of tweets is not representative, we can still use it to find potential issues we might have with the tweets' content. We will later use this analysis to decide which values to filter out of the dataset.

We start be inspecting a random sample:

In [ ]:
pd.set_option('display.max_colwidth', 140)
ex_sp_df.sample(n=10)['main']

We immediatly see that tweets containing links are not relevant to our research question (they are mostly news or adds).

In [ ]:
ex_sp_df.main[ex_sp_df.main.map(lambda x: 'http://' in x)].head(10)

Looking at the sample, we generalize this rule and make the assumption that URLs are sign that a tweet is a spam.

Using a very simplified matching research, we see that some words are problematic.

In [ ]:
pd.set_option('display.max_colwidth', 100)
ex_sp_df[ex_sp_df['main'].map(lambda x: 'suicide' in x) ]['main']

We notice that the occurences of the word _**'suicide'**_ in our set are often linked to news. This also comforts us in our choice to remove tweets with URLs in order to get better results.

In [ ]:
ex_sp_df[ex_sp_df['main'].map(lambda x: 'therapie' in x) ]['main']

Looking at the word _**'therapie'**_ (_therapy_ in German), we can confirm once again the issue with URLs.

In [ ]:
ex_sp_df[ex_sp_df['main'].map(lambda x: 'RT ' in x) ]['main'].head()

Finally, we look at the retweets to see if they can be useful. Using these results (and the findings other researchers such as the ones presented in [8]), we see that they are not useful.

<h2 id="t2">1. _Spinn3r_ Data Treatment</h2>

Now that we have familiarized with our data, we have to clean the datset. To do this, we first apply our functions locally to a small subset to make sure they work as expected. After this, we use Spark to scale up our methods and apply them to all dataset on the cluster.

### 1.1. Tweet filtering

- First, we would like to get rid of any tweets with a spam probability greater than $0.5$ (tweets more likely to be spam than not). 
- As we are only focused on the Swiss population, we only keep tweets written in the country's official languages (except for Italian as no one in our group speaks the language, meaning we won't be able to extract useful information) and in English (as most tweets in the dataset, and more generally on Twitter, are written in English)

In [ ]:
#removing spam
spam_mask = (ex_sp_df['source_spam_probability'] >= 0.5)
ex_sp_df.drop(ex_sp_df[spam_mask].index, inplace=True)
ex_sp_df.reset_index(drop=True, inplace=True)

In [ ]:
#filtering by language
lang_mask = ~ex_sp_df.lang.isin(['de', 'en', 'fr'])
ex_sp_df.drop(ex_sp_df[lang_mask].index, inplace=True)
ex_sp_df.reset_index(drop=True, inplace=True)

### 1.2. Column formatting

To start, we format the column containing the dates to make the temporal analysis easier.

In [ ]:
#convert dates to datetime format
ex_sp_df['published'] = pd.to_datetime(ex_sp_df['published'])
ex_sp_df['published'].head()

As the geo_point data is not standardised, we need to treat both types of geolocation encodings seperately.

As the example dataset does not contain the 'geo_point' field, we just present the function we will apply to do this here.

In [ ]:
def process_location_data(df):
    """preprocesses geo data in format as found on cluster
    df: dataframe, locations should be in 'geo_point' column
    """
    #get lines containing geopoint
    non_null = df[~df.geo_point.isnull()]
    
    #location separated by comma format: [longitude, latitude ]
    comma = non_null[non_null.geo_point.str.contains(',')].geo_point.str.split(pat=',', expand=True).applymap(float)
    
    #locations just next to each other [longitude latitude]
    no_comma = non_null[~non_null.geo_point.str.contains(',')].geo_point.str.split(expand=True).applymap(float)
    return pd.concat([comma, no_comma])

### 1.3. Text treatment

Following this mild cleaning, we need to work on the main subject of our analysis: the content of the tweets.

We put all caracters in lowercase (which will allow us to compare the tweets more easily without any case-sensitive-related issues).

In [ ]:
ex_sp_df['main'] = ex_sp_df['main'].str.lower()
ex_sp_df['main'].head()

As we have seen in _Part 1_, URLs are highly linked to spam (except for URLs linking pictures to the tweet). Yet, we also remove all _pic.twitter.com_ URLS as we will not use them either and delete all mentions of the format _"@user"_.

In [ ]:
url_mask = ex_sp_df['main'].str.contains("www") | ex_sp_df['main'].str.contains("http")
ex_sp_df['main'] = ex_sp_df['main'].str.replace("pic.twitter\S+", '')
ex_sp_df.drop(ex_sp_df[url_mask].index, inplace=True)
ex_sp_df.reset_index(drop=True, inplace=True)
ex_sp_df['main'].head()

To finish the text treatment, we delete all non-alphanumeric characters.

In [ ]:
alphanum_filter = lambda data: re.sub(r'[^\w\s]', '', data)

In [ ]:
ex_sp_df['main'] = ex_sp_df['main'].map(alphanum_filter)
ex_sp_df['main'] = ex_sp_df['main'].str.replace('_', '') #not removed by previous regex
ex_sp_df.main.head()

### 1.4. Spark version

After performing all these filtering and cleaning steps, the size of the set is **significantly reduced** (the new file only weights **2.4MB** instead of the **55MB** of the original file, allowing us to reduce the set by **30 times**). This is particularly helpful as the whole dataset represents around **30GB**.

_Note:_ the code was written as _**Raw NBConvert**_ as it is not meant to be run on the notebook but on the cluster. The code is only here as information for the reader, it's executable can be found in the run.py script.

We start by importing the necessary elements and defining Spark environment variables.

We then import our data.

We filter tweets using the same method.

Text treatment is a little longer as it is more thorough in Spark.

Finally, we store the data in the _JSON_ we will be using.

### 1.5. Natural Language Processing

Using  Natural Language Processing _(NLP)_ is necessary given that we work with tweets.
We only perform a simple processing pipeline (tokenization and stemming) using the goto NLP python library _**nltk**_. This is useful because we work with tweets from different languages.
We do not use stop word removal at this stage, as we will see that personal pronouns, which would be removed by stop word removal, play an important part in the detection of tweets using out dictonary.



_Note:_ once again, the code was written as _**Raw NBConvert**_ as it takes a long time to run on the dataset (**5+** minutes). The code is only here as information for the reader, it's executable can be found in the *processing_pipeline.py* script.

#### 1.5.1. Helper functions
(cf. _Next part_ below for information)

#### 1.5.2. Tweet formatting

NLP is mainly based on treating the data to obtain an adequate format to work with. It usually follows the 3 steps explicited below:
- **Tokenization:** separating words, we use a special tweettokenizer to work on the tweets
- **Stemming:** keeping only the radical of a word (allows better comparison)

<h2 id="t3">2. Spinn3r Dataset Description</h2>

To be able to provide sound answers to our research questions, we need to analyze our entire dataset to compare it to our final results.

_Note :_ we used the *processing_pipeline.py* for speed and convenience. The results were saved to the *processed_tweets* pkl file which can be found in the _'data/spinn3_tweets'_ folder.

In [ ]:
sp_df = pkl.load(open(SP_PROC_TWEETS, "rb" ))

### 2.1. General statistics:

In [ ]:
sp_df.count()

We have a total of **3936084** tweets but we note that less than half of them have *geo\_point* values attached to them.

### 2.2. Distribution of categorical data

In [ ]:
#creating pie charts to show distribution
plt.figure(figsize=(15, 15))

plt.subplot(3,3,1)
sp_df['lang'].value_counts().plot(kind='pie', colors=colors, autopct='%.2f%%')
plt.title('Distribution of language in tweets')

plt.subplot(3,3,2)
sp_df['author_gender'].value_counts().plot(kind='pie', colors=colors, autopct='%.2f%%')
plt.axis('equal')
plt.title('Distribution of gender in tweets')

plt.subplot(3,3,3)
sp_df['sentiment'].value_counts().plot(kind='pie', colors=colors,autopct='%.2f%%')
plt.axis('equal')
plt.title('Distribution of sentiment analysis results in tweets')

1. Almost half of all tweets are in English, French being the next most popular language. This is very interesting because it is quite unexpected given we are looking at tweets which were posted in Switzerland where German is the the most widely used official language with $66\%$ of the country using German, with French speakers making up $22.7\%$. [source](https://en.wikipedia.org/wiki/Languages_of_Switzerland)

2. Most accounts do not contain information on the user's gender (meaning there is no way we can have an unbiased set). However, the dataset still provides the gender of over a third of profiles (even though male and female users are not equally represented), possibly allowing to get some insights on the differences in mental distress expression between the genders.

3. Around $75\%$ of tweets are labeled as being _**NEUTRAL**_, and only $8\%$ are labeled as negative. Taking these labels at face value, this could already indicate that people on twitter do not discuss negative topics – mental health being such a topic – a lot. 

### 2.3. Distribution of temporal data

In [ ]:
#Helper functions
def get_time_distribution(times):
    publishing_time = times.map(lambda x: x.hour).value_counts() / times.count()
    publishing_time.sort_index(inplace=True)
    return publishing_time

def plot_daily(df):
    times = get_time_distribution(df['published'])
    return plt.plot(times)


def get_monthly_distribution(times):
    return times.map(lambda x: x.month).value_counts()
    
    
    
def plot_yearly(df):
    year = get_monthly_distribution(df['published'])
    sns.tsplot(data=year, time=year.index)
    plt.ylabel('number of tweets')
    plt.title('Distribution of tweets over a 10 months period')
    plt.xticks(range(1, 11),['jan', 'feb', 'mar', 'apr', 'mai', 'jun', 'jul', 'aug', 'sep', 'oct'])
    plt.xlabel('month')
    #plt.show()
    

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2,2,1)
plot_daily(sp_df)
plt.xlabel('hour')
plt.ylabel('number of tweets')
plt.title('Distribution of tweets over 24 hours')


plt.subplot(2,2,2)
plot_yearly(sp_df)

We first look at the daily pattern of data. The highest number of posted tweets is around 8pm (a number decreasing through the night with the lowest pic attained at 3am). Another low point is around lunch time.

When looking at the number of tweets posted throughout the 10 months, we see that the dataset is not balanced (a plausible explanation is the rapidly-increasing number of Twitter users). This could also be linked to the Spinn3r's tweets retrieval method.

### 2.4. Distribution of location data

As previously noted, only $\sim40\%$ of tweets are a geo-localised. However, we can still look at their overall distribution. We use [this tutorial](http://andrewgaidus.com/leaflet_webmaps_python/) (and [the geodata](https://github.com/interactivethings/swiss-maps) provided in the $3^{rd}$ homework) to create our map. We then convert our data into a shapefile using [an online converter](https://mygeodata.cloud/converter/shp-to-geojson).

_Note:_ some points are not located in Switzerland but we do not filter out the attached tweets (we suppose the dataset is correct).

We first match the shapefile format with our location data.

In [ ]:
locations = process_location_data(sp_df)
location_geo = gpd.GeoSeries(locations.apply(lambda z: Point(z[1], z[0]), 1), crs={'init': 'epsg:4326'})
location = gpd.GeoDataFrame(sp_df[~sp_df.geo_point.isnull()], geometry=location_geo) #merged data –useful later on–

In [ ]:
municipalities = gpd.read_file('data/geodata/ch-municipalities.shp') #load data
municipalities.geometry = municipalities.geometry.to_crs(location_geo.crs) #change format

Using _**gdp**_'s library, we find occurences per municipality and _'beautify'_ the data display.

In [ ]:
#join between region and location
counts = gpd.tools.sjoin(location, municipalities).groupby('id').size()

In [ ]:
#add counts to the regions
municipalities.set_index('id', inplace=True)
municipalities['counts'] = counts
municipalities.fillna(0, inplace=True) #if we can't find match we have 0 tweets
municipalities.reset_index(inplace=True)

We display using 2 layers to get more granularity (we plot the municipalities with less than 10 tweets on a different layer).

In [ ]:
municipalities_0 = municipalities[municipalities.counts < 10]
municipalities_1 = municipalities[municipalities.counts >= 10]

Using this data, we create our map.

In [ ]:
SWISS_COORD = [46.85, 8.23]
swiss_map = folium.Map(location = SWISS_COORD, zoom_start = 8, tiles = 'cartodbpositron')

#layer with municipalities containing more than 10 tweets
swiss_map.choropleth(municipalities_1.to_json(), data = municipalities_1,
                     key_on = 'feature.properties.{}'.format('id'),
                    columns = ['id', 'counts'], fill_color = 'PuRd',
                     threshold_scale=[11, 500, 1000, 2000, 5000, 10000],
                     legend_name='Number of tweets', fill_opacity=0.9)
#layer with municipalities containing less than 10 tweets in total
swiss_map.choropleth(municipalities_0.to_json(), data = municipalities_0, 
                     key_on = 'feature.properties.{}'.format('id'),
                     columns = ['id', 'counts'], fill_color = 'PuBu', 
                     threshold_scale=[0, 10], legend_name='Area of little to no data',
                     fill_opacity=0.1, line_opacity=0.3)

As the map is quite large and does not display well directly we save our map to an HTML file so it an be viewed in the widget below or independently.

In [ ]:
swiss_map.save('municipalities_map.html')

In [ ]:
%%HTML
<iframe width='900' height="500" src="municipalities_map.html"></iframe>

We see that tweets are strongly concentrated in urban centers.
This will have to be taken into consideration when looking at the Röstigraben and urban/rural differences.

<h2 id="t4">3. Dictionary Construction</h2>

### 3.1. Building the dictionary

The first step in building our dictionary was research (we use dictionnaries shown in [2] and [3] as they were used for similar problems). As we work with tweets in multiple languages, we translate them carefuly into French and German. We decided to add multiple appropriate synonyms in the translation for words that have many synonyms (as is the case with German). 

In [ ]:
dict_1 = pd.read_csv(DICT_1)
dict_1.head(10)

In [ ]:
dict_2 = pd.read_csv(DICT_2)
dict_2.head(10)

We apply our cleaning pipeline to the terms in the dictionary so that they can be matched with our processed tweets.

_Note :_ we only show the pipeline application to the $1^{st}$ dictionary as an example. The actual cleaning (and matching of keywords) is done in the *dict_filtering.py* script and saved to a pkl file (stored in the *'data/spinn3r_tweets/keyworded_tweets'* folder).

In [ ]:
en_dict = dict_2['english'].dropna()
fr_dict = dict_2['french'].dropna()
de_dict = dict_2['german'].dropna()

In [ ]:
tknzr = TweetTokenizer()
en_dict = en_dict.map(lambda x: tknzr.tokenize(x))
fr_dict = fr_dict.map(lambda x: tknzr.tokenize(x))
de_dict = de_dict.map(lambda x: tknzr.tokenize(x))

After cleaning and tokenizing, the dictionaries look like this:

In [ ]:
en_dict.head()

### 3.2. Processing the dictionary

Once we have our (clean) data, we simply run all the previous NLP methods on our dictionaries. We introduce a remove_stops method that will be used later, when building or LDA topic model.

In [ ]:
def dict_remove_stops(dictionary, language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'

    lang_set = stopwords.words(language)
    return dictionary.apply(lambda expression: [word for word in expression if word not in lang_set])

def dict_stem_words(dictionary, language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'

    stemmer = SnowballStemmer(language)

    return dictionary.apply(lambda expression: [stemmer.stem(word) for word in expression])

In [ ]:
en_dict = dict_stem_words(en_dict, 'english')
fr_dict =  dict_stem_words(fr_dict, 'french')
de_dict =  dict_stem_words(de_dict, 'german')

After stemming, the words now look like this:

In [ ]:
en_dict.head()

### 3.3 Finding keyword matches

We define to following helper functions to find matches:

In [ ]:
def compare(s, t):
    """compares to lists, and returns 
    true for all possible permutations of same list
    s, t: lists to compare
    """
    return Counter(s) == Counter(t)

def match_dict(tweet, dict_):
    """returns keywords that match in string
    tweet: tweet to find keywords in
    dict_: list of keywords
    """
    doc = [sublist for sublist in dict_ if compare(list(filter(lambda x: x in tweet, sublist)), sublist) ]
    return doc #at least one match

The match_dict function works on the dictionary (list of lists) and a single tweet (list), which makes it quite time consuming. However, working with this data structure has the great benefit of finding matches for a group of  words (e.g. _"killing myself"_), even if they are separated by a word –such as 'feeling very blue'– and dealing with words in an inverted order, which may occur due to grammatical reasons in german.

Now that we have our dictionaries, we filter the tweets to keep the ones where keyword occurs at least once in the tweets.

To find matches, we apply a mapping in the following way:

_Note:_ the actual algorithm was performed using *the dict_filtering_small.py* script and the results were saved to pickle files in the *'data/spinn3r_tweets/keyworded_tweets'* folder.

### 3.3.1. Dictonary 1


We define the following function to look at keyword occurences and coocurrences:

In [ ]:
def plot_occurences(keywords):
    """
    takes keywords and build frequency barplot
    keywords: Series containing list of keywords occuring in each tweet (list must not be empty)
    """
    keywords.map(lambda x: ' '.join(x[0])).value_counts().plot(kind='bar')
    plt.title('Keyword occurences')
    plt.ylabel('number of tweets')
    plt.show()
    
def get_keyword_stats(df):
    filtered = df[(df.keywords.map(lambda x: x != [] ))]
    print('number of matching tweets: ', filtered.main.count())
    print('percentage of matching tweets: ', filtered.main.count() / df.main.count())
    print('Some lines from the dataframe: ')
    return filtered

We decide to use a coocurrence graph (as opposed to coocurrence matrix) to visualize the keywords that occur frequently together in tweets as the matrix is very large and sparse, not allowing us to see any results, whereas the network deals better with this sparcity. To construct our graphs, we used a circular plot layout where the nodes (cooccurring words) were linked between them if they were present in the same tweet (the number of cooccurrences being represented by the edge's width).

In [ ]:
def get_coocurrences(filtered):
    #get words that have more than one occurence
    return filtered[filtered.keywords.map(lambda x: 3 >= len(x) > 1)].keywords.map(lambda x: [' '.join(sub) for sub in x])

def draw_coocurences(cooccurences):
    """
    Builds Coocurence network based on list of list of keywords found in tweets
    """
    dictionary = corpora.Dictionary(cooccurences)
    #transfrom coocurences to edgepairs for easy usage
    edges = cooccurences.map(lambda x: [dictionary.token2id[sub] for sub in x])

    lists = dictionary.token2id.keys()
    G = nx.Graph()
    G.add_nodes_from(lists)

    #add edges to graph
    for edge in cooccurences:
        if(G.has_edge(edge[0], edge[1])):
    #increase weight on graph for edges that occur more than once
            weight = G[edge[0]][edge[1]]['weight']
            G.add_edge(edge[0], edge[1], weight = 1+weight)
        else:
            G.add_edge(edge[0], edge[1], weight = 1)

    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    #scale size of graph in relation to number of words
    size = len(lists)/2
    plt.figure(figsize=(size, size))
    nx.draw_circular(G, with_labels=True, width=(weights), node_size=2000, node_color='w')
    plt.title('Coocurence network of dictionary keywords')

We now look at what kind of results our filtering provides:

**English results :**

In [ ]:
df = pkl.load(open(SP_SMALL_EN_KW, "rb" ))
small_english_filtered = get_keyword_stats(df)
small_english_filtered[['main', 'keywords']].head()

We see that we get very few results using this dictionary. Less than $0.2\%$ of all tweets match at least one keyword. This is not surprising as these are quite specialized terms.

Even though some tweets indeed show signs of mental health issues, some are completely unrelated to mental health. This discussion will be explicited in the Dictionary Evaluation (Parts _3.4_ and _3.5_).

Despite the few matches, we get false positives which indicates that the total number of tweets actually showing signs of mental distress is even smaller.

In [ ]:
plot_occurences(small_english_filtered.keywords)

The 3 most frequent keywords are _depress_, _addict_ and _suicid_, which are often casually used compared to the 5 other words we see in the chart.

In [ ]:
cooccurences = get_coocurrences(small_english_filtered)
draw_coocurences(cooccurences)

We see that depression and suicide seem to be frequently coocurring words. This makes sense, as it is depressed people that commit suicide.

**French results :**

In [ ]:
df = pkl.load(open(SP_SMALL_FR_KW, "rb" ))

small_french_filtered = get_keyword_stats(df)
small_french_filtered[['main', 'keywords']].head()

We notice that there are 3 times less matching tweets than in English, which can partially be explained by the fact that the ratio of matching tweets is not the same in both languages. Looking at the percentage of matching tweets we see that only $0.09\%$ of all French tweets contain at least one match.

In [ ]:
plot_occurences(small_french_filtered.keywords)

When it comes to French, the keyword _'suicid'_ is by far the most frequent, which all other words being used quite infrequently, having less than 100 matching tweets each.

In [ ]:
cooccurences = get_coocurrences(small_french_filtered)
draw_coocurences(cooccurences)

Despite the relative sparcity of the word depressed, we again find the coocurence of depressed and suicde in french.

**German results :**

In [ ]:
df = pkl.load(open(SP_SMALL_DE_KW, "rb" ))
small_german_filtered = get_keyword_stats(df)
small_german_filtered[['main', 'keywords']].head()

In German, the number of matching tweets is even lower, despite having used synonyms to compensate for the multitude of possible matching words.

In [ ]:
plot_occurences(small_german_filtered.keywords)

Looking at the keyword frequencies, we see that similar to english, addiction, depression and suicide ['abhang', 'depression', 'suizid', 'selbstmord'] are the most frequent keywords, which the other keywords in the dictionary having much fewer matches. 

In [ ]:
cooccurences = get_coocurrences(small_german_filtered)
draw_coocurences(cooccurences)

German also shows a connection between suicide and depression, just like French and English.

### 3.3.2. Dictionary 2


We now look at the second dictionary retrived from previous research. This dictionary contains a broader range of words, and is a superset of the previous dictionary. It contains a more generalized list of words, which let us to believe that it may induce more false positives.

**English results :**

In [ ]:
df = pkl.load(open(SP_EN_KW, "rb" ))
english_filtered = get_keyword_stats(df)
english_filtered[['main', 'keywords']].head()

At first glance we can see that this dictionary gives us many more matches, with about $1\%$ of all english tweets containing at least one keyword, this is about five times more than for dictionary 1. We see that the matches tend to be more general about feelings and less about specific illnesses. 

In [ ]:
plot_occurences(english_filtered.keywords)

Even though not all tweets express distress (just as before), we can clearly see sadness, loneliness or fear.

In [ ]:
cooccurences = get_coocurrences(english_filtered)
draw_coocurences(cooccurences)

We note the strong connections between lonely, sad, depressed, distressed and anxiety.

**French results :**

For french we find similar results as for english. We have more matches, and these are less specific to mental illness:

In [ ]:
df = pkl.load(open(SP_FR_KW, "rb" ))
french_filtered = get_keyword_stats(df)
french_filtered[['main', 'keywords']].head()

In [ ]:
plot_occurences(french_filtered.keywords)

Looking at the words we see that 'kill me' is by far the most frequent, followed by words like 'sadness' and 'suicide'. Gives results quite different form the ones we find for English.

The owerwhelming frequency of 'kill me' might be due to the casual usage of kill me in french, as not to indicate mental distress but annoyance.

In [ ]:
cooccurences = get_coocurrences(french_filtered)
draw_coocurences(cooccurences)

While we previously indicated that the frequent occurence of kill me might be due to casual usage, we here see that kill me co-occurs with 'I hate myself' quite frequently indicateing to us that we can also find a lot of serious occurences of the phrase kill me in french.

**German results :**

German also shows an increase in matches with the larger dictionary. However, due to the number of matches being small originally, the total number of matches is still quite small.

In [ ]:
df = pkl.load(open(SP_DE_KW, "rb" ))
german_filtered = 
german_filtered[['main', 'keywords']].head()

In [ ]:
plot_occurences(german_filtered.keywords)

We note that the most frequent words are again 'sad', followed by expressions of loneliness and emptiness. This distribution is similar to the distribution of french words.

In [ ]:
cooccurences = get_coocurrences(german_filtered)
draw_coocurences(cooccurences)

In the coocurence matrix for this dictionary, the strongest connection if formed by lonely and empty.

### 3.4. Dictionary evaluation (using our sample labeling process)

After crossing the dataset with our second dictionary to retrieve tweets exhibiting _mental distress_, we labeled a sample to see if the tweets indeed show such signs. The sample was retrived using the .sample function, saved to a csv file. We then labeled the tweets in this csv file.

Note that this task is complex as it is very subjective (choosing tweets displaying mental health issues was done at the discretion of the person labeling them). Our main conclusion from this step was the necessity to "expand" our research to tweets showing signs of mental distress in general and not only mentioning mental illnesses.

Tweets were labeled (binary labeling, 0 or 1) "losely" using multiple signs of mental distress, mostly sadness. Tweets with the following specificities were labeled as showing mental distress: 
    - nostalgia (either for the past or simply for the end of a nice day)
    - mention of sad activities (people who watched sad movies, listened to sad songs as this was most probably triggered by a previous feeling of sadness)
    - mention of being mad (either over other people like haters or, more often, mad with their families)

#### 3.4.0. Helper functions


We define the following helper functions:

In [ ]:
def tweet_search(df, word):
    """
    This retrieves all tweets of a DataFrame 'df' mentioning a word 'word'
    """
    return df[df['main'].map(lambda x: word in x)]['main']

In [ ]:
def md_search(df, word, col):
    """
    This functions shows the relative appearance of words in our dictionary to show that they should be taken out.
    """
    md_count = df[df[col].map(lambda x: word in x) & df['mental'] == 1][col].count()
    print("Tweets showing mental distress and containing the word '" + word + "': " , md_count)
    count = df[df[col].map(lambda x: word in x)][col].count()
    print("Total appearance of the word '" + word + "': " , count)
    print("Ratio between these 2 numbers : ", md_count/count)

In [ ]:
def blacklisted_search(df, word):
    """
    This function shows the total appearance of words to show why they should be blacklisted.
    """
    count = df[df['split'].map(lambda x: word in x)]['split'].count()
    print("Total appearance of '" + word + "': ", count)

In [ ]:
first_iter = pd.read_csv(SP_EN_LBL)
first_iter.main.head()

#### 3.4.1. Key _(qualitative)_ takeaways form the labeling
 

1. Some tweets mentioned the mental distress of other people (either how they helped them or raising awareness over these issues)
2. Some tweets showed desperation (a form of mental distress) over the "way the world is", these were not included.

3. An unexpected insight (which we decided not to pursue further as it was too tenuous) was the link between summer and mental distress. Indeed, most of tweets mentioning summer showed some sadness due to the absence of friends (from university for example).
4. Some tweets only included _"motivational"_ quotes. We can suppose that these reflect bad times. Nonetheless, these were not included as our previous assumption is a little far-fetched. Moreover, this could have messed with our model (they did not include specific words of our dictionary nor specific mentions to mental distress).

5. A lot of mental distress tweets are related to unilateral feelings. They are often messages specifically targeted at someone who most certainly doesn't know of their existence (for example work collegues or college classmates).

6. Most tweets talking about depression or terrible low self-esteem show _« covert-humour »_, a coping technique used to hide ones self-deprecation. These tweets are very difficult to detect as they use sarcasm (which can not be perceived using current NLP techniques) and seldom use words from our dictionary.

#### 3.4.2. Key _(quantitative)_ takeaways

After the labeling, we found the necessity of both changing our current dictionary through additions as well as removals. The removal part is done by creating a new dictionary, which we decided to call a _negative_ dictionary, a blacklist of words used to exclude tweets.

**Words to include in our dictionary :**

In [ ]:
tweet_search(first_iter, 'unfortunately')

In [ ]:
tweet_search(first_iter, 'therapy')

In [ ]:
tweet_search(first_iter, 'overthinking')

In [ ]:
tweet_search(first_iter, 'breakdown')

Even though these expressions do not appear very often, they are unique to tweets showing mental distress. Thus, we decide to include them in our new dictionary.

In addtion we add the expressions: _panic attack_, _sleepless_, _problems falling asleep_, _breakdown_, and _mental issue_.

**Words to remove from our dictionary :**

In [ ]:
md_search(first_iter, 'addict', 'split')

Even though  (_addict_), was added to the dictionary by a proffesional who surely must have had good reason to include it, we decided to demonstrate our method only on a few. When looking at the number, we see that more than half the tweets containing the (previously) specified words are false positives, which drives us to strike them from our dictionary.

**Words to blacklist :**

In [ ]:
blacklisted_search(first_iter, 'overw')
blacklisted_search(first_iter, 'overwatch')

In [ ]:
blacklisted_search(first_iter, 'syria')
blacklisted_search(first_iter, 'assad')
blacklisted_search(first_iter, 'asylum')

In [ ]:
blacklisted_search(first_iter, 'trump')
blacklisted_search(first_iter, 'clinton')
blacklisted_search(first_iter, 'america')
blacklisted_search(first_iter, 'usa')

As we can see, the words we want to strike represent major recurring themes (mostly having to do with the news) which do not have anything to do with mental distress (none of these tweets were labeled as showing signs of mental distress). Once again, we only showed our idea on a small subset of topics, the list of words we want to blacklist is : _gov_, _government_, _syria_, _assad_, _refugees_, _asylum_, _overwatch_, _vine_, _trump_, _hillary_, _america_, _usa_ and _india_. 

_Note:_ two other expressions are blacklisted, but that is due to our first research algorithm (which did not match perfectly the strings) : _suicidesquad_ (contains _suicide_) and _spain_ (contains _pain_)

#### 3.4.3. French and German labeling

When labeling the other languages, we saw that only $\sim10\%$ of tweets containing at least a keyword from a dictionary actually showed mental distress. 
This result may be due to a multitude of factors. On one hand linguistic differences may make simple translation impossible and improving the dictionnaries would probably give better results, but as no research of this kind has been done in these languages, we can only improve our results through many iterations (and labeling steps). 
Additionaly, lemmatizing may provide better results for french and german, but there are currently no libraries that offer this reliably. Natural Language Processing for languages other than English seems not to be very well developped, providing us with little ways to imporove our NLP pipeline to get better matchigs for french and german.

For these reasons, we decided to drop both languages and concentrate on English.

### 3.5. Dictionary evaluation (using LDA)

Once again, we only work with tweets which were crossed with our second dictionary. As this is applied on a larger scale than our labeling, it allows us to show subjects which should be avoided (as showed when constructing our _negative_ dictionary).

We confirm the results found in the labeling and add additional blacklisted words.

_Note :_ we use a script we build our LDA models on and save them to the _'data/models'_ folder. We will quickly present these models we created but our focus will be on understanding them (and displaying our results using the _**pyLDAvis**_ library).

#### 3.5.1. Building an English LDA model

To build our LDA model, we need to create a dictionary and define 'texts' (in our case the tweets). This model is then passed to gensim, which then implements the proper LDA model construction. The number of topics was chose was of a size such that a human could interpret them easily. The number of passes and iterations were chosen to be as large as possible, as the corpus we are working on is relatively small.

We first remove stopwords from the filtered dict to get better results. Due to stemming we may miss some matches, but looking at the results we see that the stop word removal still impoves the topic model.

In [ ]:
english_stop_rem = dict_remove_stops(english_filtered.tokenized, 'english')

We from the dictionary and corpus with the stop word removed tweets:

In [ ]:
dictionary = corpora.Dictionary(english_stop_rem)
dictionary.filter_extremes(no_below=2, no_above=.3) #removes too frequent / sparse words
texts = english_stop_rem.tolist()
corpus = [dictionary.doc2bow(text) for text in texts]

Model creation is done once and then saved, we thus provide the code in RawNBconvert format.

In [ ]:
ldamodel = models.LdaModel.load('models/english.model') #load the model

We create the following visualisation on order to inspect the topics created by the model:

In [ ]:
#Creating the visualisation
vis_data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis_data)

We use the $\lambda$ slider to try to find human understandble topic description.

We find that topic 4 contains a lot of words that we would like to exclude. By setting $\lambda$ 0.6 we find keywords such as suicide & squad, indicating that the 2016 movie suicide squads is influencing our findings.

We also see words such a hillary, trump, syria and bomb, indicating that this topic aggregates news topics that were prominent in 2016. This gives us more security in our findings from manual labeling. The remaining clusters to not contain words that stand out, they seem to be a general aggregation of words relating to feelings.


**Words to add to our dictionary :**

Once again, we use a different format to show a special expression. We need to delete _'Sonic Mania'_ as it contains the word _'mania'_ (which couldn't be taken out even with our new search algorithm). However, we can see that affining our dictionary worked as we have a lot less words to blacklist :  _stress relief_ and words with the radical _"addict"_.

<h2 id="t5">4. Spinn3r Dataset Analysis:</h2>

We process the final dictionary using the *dict_filtering_final.py* script.


Now, let us discuss the final results from our dict.

In [ ]:
final_df = pd.read_pickle('data/spinn3r_tweets/keyword_tweets/final_english_keyworded_tweets.pkl')

In [ ]:
has_keyword = final_df.keywords.map(lambda x: len(x) > 0) #get tweets that contain at least one matchign keyword in dict

In [ ]:
keyword_tweets = get_keyword_stats(final_df)
keyword_tweets[['main', 'keywords']].head(10)

At a glance we can see that many of the matches we find are quite good!

### Keyword distribution

We now again look at how the words are distributed over all.

In [ ]:
plot_occurences(final_df[has_keyword].keywords)

We find that using this dict, the difference between the occurence frequency is less steep than before, there are no large gaps in usage frequency

In [ ]:
cooccurences = get_coocurrences(final_df[has_keyword])
draw_coocurences(cooccurences)

Looking at the coocurrence graph, we find the follwoing:

- frequently occuring words are often frequently connected, such as depress, distress and anxiety.
- interessting patterns of coocurence can be observed
    - the strong link between anxiety and panic attacks. A panic attack is a sudden surge of overwhelming anxiety and fear, making this link meaningful.
    - we see a strong link between depression and suicide.
    
    - from the above it follows that the coocurences are meaningful and not random
    - our interpretation for this coocurence is that despite that character limit on twitter, people are able to express symptoms and personal issues on twitter.

### Sentiment score

We look at the distribution of the sentiment scores over all of the dataset and compare it to the distribution of sentiment scores in the set of tweets that were labeled with at least one keyword.

In [ ]:
plt.figure(figsize=(10, 10))
plt.suptitle('Sentiment in all tweets vs sentiment in tweets containing keyword')
plt.subplot(2,2,1)
plt.ylabel('percentage of tweets')
plt.title('All tweets')
(final_df.sentiment.value_counts().sort_index()/final_df.sentiment.count()).plot(kind='pie', autopct='%.2f%%', colors=colors)

plt.subplot(2,2,2)
plt.title('Tweets with keyword match')
(final_df[has_keyword].sentiment.value_counts().sort_index()/final_df[has_keyword].sentiment.count()).plot(kind='pie', autopct='%.2f%%', colors=colors)

plt.savefig('sentiment.png')

We immediately see that sentiment labels are distributed in a completly different manner in the two dataset.

The filtered tweets, as is to be expected by the topic of filtering at hand, has an overall much higher percentage of tweets labeled as negative. Interesstingly we find about $13.36\%$ to be labeled as positive. We will later examine such positive tweets.  

### Gender

In [ ]:
plt.figure(figsize=(10, 10))

plt.suptitle('Gender distribution all tweets vs in tweets containing keyword')
plt.subplot(2,2,1)
plt.ylabel('percentage of tweets')
plt.title('All tweets')
(final_df.author_gender.value_counts().sort_index()/final_df.author_gender.count()).plot(kind='pie', autopct='%.2f%%', colors=colors)
plt.subplot(2,2,2)
plt.title('Tweets with keyword match')
(final_df[has_keyword].author_gender.value_counts().sort_index()/final_df[has_keyword].author_gender.count()).plot(kind='pie', autopct='%.2f%%', colors=colors)
plt.show()

Unlike sentiment, the distribution of gender across the two datasets is almost identical. While there is a 3 percent difference between male and female, we find that given the large amount of unknowns in the dataset, this well be due to chance.

An alternative theory we developed, taking into consideration our observations made during the manual lebeling, is that the difference may be due to the (very high) number of tweets by _"fangirls"_, female tweet authors mentioning their love for their favorite music bands (mostly Asian) or one of their favorite actors.

### User analysis


Havign kept the userIds, we are now abel to see who is writing these tweets. First we look at how many users are responsible for the tweets we find to exhibit signs of mental distress.

In [ ]:
user_posts = final_df[has_keyword].author_user_id.value_counts()
user_posts.head()

In [ ]:
print('Percentage of users with at least one tweet containg a keyword: ', user_posts.count() / final_df.author_user_id.value_counts().count())
print('Total users ',  user_posts.count())

We can see that out of all users in the dataset, almost 5 percent had at least one tweet containing one of our keywords.

We immediatley see that 2 users are posting very many tweets that contain a keyword, with the number quickly decreasing:

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Number of tweets labeled per user')
#looking at users with more than 25 posts for visibility
ax = user_posts[user_posts > 5].plot(kind='bar')
plt.xlabel('Bar representing a user')

plt.ylabel('Number of keyword matched tweets')
plt.setp(ax.get_xticklabels(), visible=False)
plt.show()

Looking at the first graph, the tweets for which a given user is responible flattens quite fast, we have two users writing tweets ocontaining a lot of our keywords, followed by 4 still using a lot but stil comparatively less etc.

Seeing this distribution we become interessted in what these frequent tweeters are tweeting about:

We look at the posts of a user who's tweets match very many keywords:

In [ ]:
final_df[has_keyword][final_df.author_user_id == 3126239893].sort_values(by='published')['main'].sample(10)

We find that the user seems to clearly have issues with mental health.

### Temporal analysis

We now again look at differences between temporal pattern. We consider both daily and the overall patter over the year.

In [ ]:
plot_daily(final_df[has_keyword])
plot_daily(final_df)
plt.xlabel('hour')
plt.ylabel('number of tweets')
plt.legend(['keywords', 'all tweets'])
plt.title('Distribution of tweets over 24 hours')
plt.show()

We observe that the distribution line for the keyworded tweets is much less smooth. This is likely due to the small number of tweets in this set compared to the set of all tweets.

We find no deviations large enought to merit any further investigation. We consider that we can not observe a difference when it comes to daily frequency in the tweets.

In [ ]:
yearly_distribution = final_df[has_keyword].published.map(lambda x: x.month).value_counts() / final_df[has_keyword].published.count()
yearly_distribution2 = final_df.published.map(lambda x: x.month).value_counts() /  final_df.published.count()

sns.tsplot(data=yearly_distribution, time=yearly_distribution.index, color='orange')
sns.tsplot(data=yearly_distribution2, time=yearly_distribution2.index)
plt.legend(['filtered', 'overall'])
plt.ylabel('relative percetage of tweets')
plt.title('Distribution of tweets over a 10 months period')
plt.xticks(range(1, 11),['jan', 'feb', 'mar', 'apr', 'mai', 'jun', 'jul', 'aug', 'sep', 'oct'])
plt.xlabel('month')
plt.show()

Again, the percentage of tweets made on a given day does not differ in a clearly observable manner between the two sets of data. We thus say that we can not find a difference between the two datasets.

### Location analysis

We now look at location again. Using the exact same algorithm previously described above we create a map with the tweets that contain at leats one keyword in the dict, and save it to an html file.

In [ ]:
%%HTML
<iframe width='900' height="500" src="dist.html"></iframe>

This map very clearly shows the overall sparcity of the dataset. We observe hotspots around the largest cities in Switzerland -Zürich, Geneva, Lausanne and Basel. Due to the overall sparcitie we again can not draw any concrete conclusion.

<h2 id="t6">5. twitter-swisscom Data Processing Pipeline</h2>

Due to issue with the cluster, we decide to use the dataset in local and perform the necessary processing steps locally.

By removing unnecessary columns, tweets containing websites, retweets, and general gibberish (a set of characters mixed with numbers), we can reduce the size of the dataset from  5.6 GB down to 1.2 GB. The code for this process can be found in the *run_d1.py* script. We save this dataset in a .csv file for further processing.

As mentioned previously, this set is not annotated with the language of the tweet. We thus needed to use an external language classification library to label the tweets and only keep English tweets. For efficiency reasons, we use the *guess_language* function from sprit. Using this function, the labeling of all 14 million tweets took about 2 hours.

To do most of the processing, we first need to do a quick mapping between the columns of the **Spinn3r** dataset and the columns of the **twitter-swisscom dataset**, as this will permit us to use all the functions we created before again.

Thus, we have the mapping as following, in the form of **Spinn3r column** -> twitter-swisscom column:

- **author_user_id**           -> userId
- **geo_point**                -> longitude + latitude (or: placeLongitude + placeLatitude)
- **main**                     -> text
- **published**                -> createdAt
- **source_followers**         -> followersCount
- **source_following**         -> friendsCount (or is it folowwers that are following ?)
- **source_location**          -> userLocation

After this we save the tweets that were labeled as English to an other .csv file.

In [ ]:
twitter_df2 = pd.read_csv('data/twitter-swisscom/dataset2_english.csv')

We perform some simple data processing that is needed after retrieving the file from the .csv.

While working on the tweets we addictionaly found issues with malformed columns, making us drop 300'000 columns, which do not contain the correct information.

In [ ]:
twitter_df2.rename(columns={'createdAt':'published'}, inplace=True)
#get tweets that have a date, we consider tweets without dates to be garbage
twitter_df2 = twitter_df2[~twitter_df2.published.isnull()]
#transfrom to string so we can check formatting
twitter_df2['published'] = twitter_df2['published'].astype(str)
#remove deformed columns
tw = twitter_df2[twitter_df2.published.map(lambda x: len(x) == 19)]

print('number of malformed columns : ', len(twitter_df2[twitter_df2.published.map(lambda x: len(x) != 19)]))
#transform to date-time
tw.published = pd.to_datetime(tw['published'])

In [ ]:
#tw.to_pickle for later retrieving in script
#tw.to_pickle('data/twitter-swisscom/dataset2_english_clean.pkl')

We now look at how many tweets we have remaining: 

In [ ]:
tw.count()

We only have 1 million tweets remaining, which is not much given the initial size of the dataset.

From the previous dataset,we know that keywords are quite sparse, with at most $2\%$ matching. So despite this dataset spanning a longer amount of time, we still find that only very little usable tweets remain.

<h2 id="t7">6. twitter-swisscom Dataset Analysis</h2>

In [ ]:
plot_daily(tw)
plt.xlabel('hour')
plt.ylabel('number of tweets')
plt.title('Distribution of tweets over 24 hours')

The temporal distribution over one day is similar to the distrubution we observed in part 2.

We note that we can find dates as early as 2002 in the dataset:

In [ ]:
tw[tw.published.map(lambda x: int(x.year) <= 2006)].published

This must be an error, as twitter first went online in 2006. Due to the sparcity of tweets pre-2013 we only look at tweets after 2013.

In [ ]:
tw = tw[tw.published.map(lambda x: int(x.year) >= 2013)]

We now look at the yearly distribution:

In [ ]:
#convert values for plotting
yearly_distribution = tw.published.map(lambda x: x.date()).value_counts()
date = matplotlib.dates.date2num(yearly_distribution.index)
#plot
sns.tsplot(data=yearly_distribution, time=date)
plt.title('Daily tweet frequency over 3 year period')
plt.xlabel('Number of tweets')
plt.ylabel('numberical translation of day')
plt.show()

The plot we find is very odd. Again, we have no explanation for this.

Despite some of the issues with the dataset we run the keyword matching algorithm on it.

In [ ]:
final_df = pd.read_pickle('data/twitter-swisscom/final_english_keyworded_tweets.pkl')
final_df  = final_df[~final_df.keywords.isnull()]
has_keyword = final_df.keywords.map(lambda x: len(x) > 0)
cooccurences = get_coocurrences(final_df[has_keyword])
plot_occurences(final_df[has_keyword].keywords)
draw_coocurences(cooccurences)

<h2 id="t8">7. Conclusion</h2>

As we can see, tweets showing mental distress are very sparse. For this reason, it very hard to get all the information we need to answer our research questions (for example determining seasonal or temporal patterns). Thus, it is not possible to conduct studies in Swizerland of the same scope of those presented in American papers, which is why we consider our results to offer more of a qualitative approach rather than purely quantitative results.

We find that almost $5\%$ of users tweet about mental distress. However, their discussions only represent $\sim0.4\%$ of the dataset, corresponding to a total of $7152$ tweets. We were not able to see clear linguistic patterns in these tweets because of our handicap using NLP methods which were not appropriate for our research. However, we noticed some convergence in the keywords distribution and their cooccurrences between French and English MD tweets.

Unfortunately, we cannot draw any further conclusions due to the overall sparsity of the data. We cannot analyze the gender difference – they are too similar to get any statistically relevant results – or the spatio-temporal information – the time distribution is skewed and the location data only shows significant Twitter activity in 3 municipalities (i.e. \textit{Geneva}, \textit{Lausanne} and \textit{Zurich}). 

When trying to compare our results to national census data, we found that these numbers were also too sparse and approximative. This further shows the need for real research on mental health related issues in Switzerland and for the conversation to get going.

To conclude, it is possible to use dictionaries to study the expression of mental distress in tweets and to analyze their patterns. However, the sparsity of the data in Switzerland prevents us from performing a large scale research, not offering the possibility of drawing any statistically relevant conclusions. This issue can only be addressed by developing new NLP methods which can be applied to other languages than English (namely French, German and Italian) and by defining an original methodology which takes into account Switzerland’s special situation. 